<h3>Data Description</h3>
<p>
    The `admissions_processed_morphine_sulfate.csv` file is processed as follows, and is a combination of the `PRESCRIPTIONS.csv`, `ADMISSIONS.csv` and `PATIENTS.csv` files found from the MIMIC-III database,
    <ul>
        <li> There are `6618` unique patients. Each patient could have had multiple hospital stays, but we only considered the first hospital stay that the patient had. The rationale is that we wanted a first impression of the patient.
        <li> These 6618 patients comprise four ethnicities: [WHITE, BLACK, ASIAN, HISPANIC] </li>
        <li> The diagnosis that were selected for consideration were only those that were shared by all four ethnic groups, there is a distribution of these diagnostics among each group in the other jupyter notebook. </li>
        <li> Ages were calculated by taking the difference between birthdate and admittime, for ages that were negative due to HIPAA compliance, we readjusted them to all be 89. </li>
        <li> 122 covariates are considered: [age, HOSPITAL_EXPIRE_FLAG, DIAGNOSIS:%s (114 of them), hosp_duration, INSURANCE (5 types)] </li>
        <li> Only patients that were administered morphine sulfate were then considered, we looked at the total amount they were administered for their single hospital stay duration by taking the FORM_VAL_RX value of the drug.
    </ul>
</p>
<p> Covariates are described above, there are 122 of them, e.g. age and different diagnosis types </p>
<p>Treatment is done by comparing one ethnic group vs the rest, e.g. (WHITE vs [ASIAN, BLACK, HISPANIC]) or (BLACK vs [ASIAN, WHITE, HISPANIC) </p>
<p>Output is the amount of the morphine sulfate the patient is administered</p>


In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = pd.read_csv("./data/admissions_processed_morphine_sulfate.csv")

In [3]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,age,TOTAL_FORM_VAL_DISP_MAX,drug
0,10,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,MARRIED,WHITE,2178-04-15 20:46:00,2178-04-16 06:53:00,brain mass,0,1,50,1.25,Morphine Sulfate
1,12,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicaid,...,NaN,WHITE,NaN,NaN,coronary artery disease,0,1,39,2.00,Morphine Sulfate
2,18,20,157681,2183-04-28 09:45:00,2183-05-03 14:45:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,...,WIDOWED,WHITE,NaN,NaN,coronary artery disease\coronary artery bypass...,0,1,75,3.00,Morphine Sulfate
3,19,21,109451,2134-09-11 12:17:00,2134-09-24 16:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,...,MARRIED,WHITE,2134-09-11 09:22:00,2134-09-11 22:30:00,congestive heart failure,0,1,87,2.00,Morphine Sulfate
4,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,MARRIED,WHITE,NaN,NaN,coronary artery disease\coronary artery bypass...,0,1,71,0.40,Morphine Sulfate


In [4]:
def df_to_X(df):
    
    # include age and hospital expire flag
    covariates = ['age', 'HOSPITAL_EXPIRE_FLAG']
    X = df[covariates]
    
    # include onehots for diagnosis
    diagnosis = pd.get_dummies(df.DIAGNOSIS)
    diagnosis.columns = ['DIAGNOSIS:%s' %d for d in diagnosis.columns]
    X = pd.concat([X, diagnosis], axis=1)
    
    # include duration of hosptial stay
    hosp_duration = (df['DISCHTIME'].astype('datetime64[ns]') - df['ADMITTIME'].astype('datetime64[ns]')).dt.days
    X['hosp_duration'] = hosp_duration
    
    # include onehots for insurance
    insur = pd.get_dummies(df.INSURANCE)
    insur.columns = ['INSURANCE:%s' %i for i in insur.columns]
    X = pd.concat([X, insur], axis=1)  

    
    # normalize duration because it is non-categorical
    d_mu = X['hosp_duration'].mean()
    d_std = X['hosp_duration'].std()
    X['hosp_duration'] = X['hosp_duration'].apply(lambda dp: (dp-d_mu)/d_std)

    # normalize age because non-categorical
    age_mu = X['age'].mean()
    age_std = X['age'].std()
    X['age'] = X['age'].apply(lambda age: (age-age_mu)/age_std)

    return X

def df_to_T(df, eth):
    return df['ETHNICITY'].apply(lambda x: int(x==eth))

def df_to_Y(df):
    return df['TOTAL_FORM_VAL_DISP_MAX']

In [5]:
X = df_to_X(df)
T = df_to_T(df, 'WHITE')
Y = df_to_Y(df)
print('X: ', X.shape)
print("T: ", T.shape)
print("Y: ", Y.shape)

X:  (6618, 122)
T:  (6618,)
Y:  (6618,)


### Computing ATE using GBM

In [7]:
from sklearn.ensemble import GradientBoostingClassifier as GradBoost

In [8]:
# for our first ate, let's compute the average treatment effect of white vs black
# can turn this into a function for arbitrary treatment pairs later

t1 = 'WHITE'
t2 = 'BLACK'
T1 = df_to_T(df, t1)
T2 = df_to_T(df, t2)

clf1 = GradBoost(n_estimators=500).fit(X, T1)
clf2 = GradBoost(n_estimators=500).fit(X, T2)

In [34]:
# in class, the formula was (sum_{treated} weight * outcome) / n, where n is the size of the dataset
# in the paper, the formula is (sum_{treated} weight * outcome) / (sum_{treated} weight)
# gonna go with class approach here, but note that the other approach might give better results
# edit: checked and class approach gives ATE of 0.6 not 0.02

treated1 = (T1 == 1)
X1 = X[treated1]
prop_weights1 = (len(X1) / len(X)) * np.reciprocal(clf1.predict_proba(X1)[:,1])
reciprocal1 = len(X1) # np.sum(prop_weights1)
weighted_mean1 = sum(np.multiply(Y[treated1], prop_weights1)) / reciprocal1

treated2 = (T2 == 1)
X2 = X[treated2]
prop_weights2 = (len(X2) / len(X)) * np.reciprocal(clf2.predict_proba(X2)[:,1])
reciprocal2 = len(X2) # np.sum(prop_weights2)
weighted_mean2 = sum(np.multiply(Y[treated2], prop_weights2)) / reciprocal2

print(weighted_mean1)
print(weighted_mean2)
print(weighted_mean1 - weighted_mean2)

1.8825201266419151
1.2384663866570589
0.6440537399848563


### Check for Balance

In [35]:
# compute unweighted mean and standard deviation of each covariate for the pooled sample across all treatments
population_covariate_means = np.array(X.mean(axis=0))
population_covariate_stds = np.array(X.std(axis=0))

In [39]:
# compare the population that got treatment 1 after weighting to the unweighted full population
covariates1 = np.array(X1)
weights1 = prop_weights1.reshape((len(prop_weights1), 1))
weighted_covariates1 = np.multiply(covariates1, weights1)
covariate_means1 = np.array(weighted_covariates1.mean(axis=0))

PSB1 = np.divide(np.abs(covariate_means1 - population_covariate_means), population_covariate_stds)
print(PSB1)
for i in range(len(PSB1)):
    if PSB1[i] > 0.2:
        print("Bad covariate: {}".format(i))

[1.60321888e-02 8.47479026e-03 8.35842982e-04 1.18535073e-03
 5.16191198e-03 9.91582990e-04 8.59713595e-04 9.41973877e-03
 6.34580531e-05 9.40429077e-04 4.83865208e-03 1.78153512e-03
 4.22275995e-05 2.60156732e-03 9.99275582e-04 6.48224316e-03
 9.54736680e-03 1.67684386e-03 6.20422258e-04 1.39607906e-02
 1.10936254e-03 6.69492564e-04 4.03534707e-03 5.46936920e-04
 2.53707341e-03 8.29839167e-04 9.70348134e-04 3.84145772e-03
 3.99788613e-03 9.65209351e-04 1.98124536e-03 2.92320906e-03
 5.31122520e-03 7.06114232e-03 1.79025199e-04 3.99011593e-04
 1.17359535e-03 7.32053802e-04 3.79390686e-03 1.71423554e-03
 4.02252581e-03 3.33275056e-03 5.05612408e-03 1.65811162e-03
 6.35087613e-03 1.06633612e-02 1.40912944e-03 8.17437660e-05
 7.63894656e-03 1.44928735e-03 4.98228517e-03 8.97553540e-03
 8.61870356e-03 7.56761003e-03 3.62700118e-04 2.14182354e-02
 9.08248978e-03 6.06775854e-03 1.73124549e-04 5.67651115e-03
 2.56895083e-04 1.49627297e-03 1.53743977e-03 2.81387211e-03
 1.39956266e-03 7.889227

In [40]:
# compare the population that got treatment 2 after weighting to the unweighted full population
covariates2 = np.array(X2)
weights2 = prop_weights2.reshape((len(prop_weights2), 1))
weighted_covariates2 = np.multiply(covariates2, weights2)
covariate_means2 = np.array(weighted_covariates2.mean(axis=0))

PSB2 = np.divide(np.abs(covariate_means2 - population_covariate_means), population_covariate_stds)
print(PSB2)
for i in range(len(PSB2)):
    if PSB2[i] > 0.2:
        print("Bad covariate: {}".format(i))

[0.00887811 0.16687318 0.04113148 0.07201849 0.03923915 0.04366594
 0.02281499 0.04201548 0.10188283 0.0253638  0.00796156 0.04409656
 0.0476587  0.0407712  0.07506617 0.04164326 0.03019012 0.042238
 0.03092021 0.03399998 0.0475128  0.0641471  0.03011024 0.01750461
 0.06250759 0.03253738 0.04231953 0.00669955 0.02268606 0.09564386
 0.02140831 0.02056774 0.03573916 0.01926563 0.06622356 0.10778558
 0.09186391 0.06913081 0.02449357 0.04339121 0.01956216 0.03889844
 0.00361141 0.02749495 0.03396597 0.04364838 0.04261755 0.01035993
 0.02749495 0.0346337  0.04427798 0.0378232  0.03012149 0.02722087
 0.02080404 0.00377027 0.01817792 0.02129428 0.01863838 0.03251745
 0.05505237 0.06277226 0.04603919 0.05770339 0.02062878 0.03535976
 0.03672192 0.06476835 0.01950885 0.02145259 0.05297328 0.0225121
 0.03689951 0.08100076 0.05905052 0.05803824 0.04603919 0.04825044
 0.04922542 0.0209026  0.00588785 0.01738539 0.0221898  0.04070665
 0.02896558 0.03689951 0.00687467 0.05074424 0.00788743 0.0117241